In [66]:
import selenium 
from selenium import webdriver
import os
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys

In [67]:
s_time = time.time()
key_word = input('업체를 조회할 키워드를 입력하세요 : ')
cnt = int(input('총 몇 건의 데이터를 조회하시겠습니까? : '))
f_dir = input('파일 저장경로는? : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+key_word)
os.chdir(f_dir+s+'-'+key_word)

f_name = s+'-'+key_word+'.txt'
fc_name = s+'-'+key_word+'.csv'
fx_name = s+'-'+key_word+'.xlsx'


path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
driver = webdriver.Chrome(path, options=options)

driver.get('https://www.naver.com/')
time.sleep(5)

driver.find_element_by_id('query').send_keys(key_word)
driver.find_element_by_id('query').send_keys('\n')
time.sleep(5)

#결과 더 보기 버튼 클릭
driver.find_element_by_class_name('api_more').click()
time.sleep(5)


no = 1
f = open(f_name, 'a', encoding='utf-8')
title_list = []
desc_list = []
juso_list = []
phone_list = []
jong_list = []
page_list = []
grade_list = []
review_list = []

#while문으로 페이지 변경 전체 반복 1번 걸기 
while True : 
    driver.switch_to.default_content()
    #항목반복 2번 걸기 
    for x in range(50) : 
        driver.switch_to.frame('searchIframe') #
        try : 
            driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[1]/ul/li[%s]/div[1]/a/div[1]/div/span[1]' %x).click()
            time.sleep(5)
        except : 
            try : 
                driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[1]/ul/li[%s]/div[1]/a/div[1]/div/span' %x).click()
                time.sleep(5)
            except : 
                break
        driver.switch_to.default_content() #
        
        driver.swith_to.frame('entryIframe')
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        print('-'*100)
        f.write('-'*100+'\n')
        
        title = soup.find('span', '_1_hm2').find('span', '_3XamX').get_text().replace('\n',"").strip()
        title_list.append(title)
        print('1.업체명 : %s' %title)
        f.write('1.업체명 : %s'%title+'\n')
        
        try : 
            desc = soup.find('div', 'rZaqA MFyRY').get_text().replace('\n', "").strip()
            desc_list.append(desc)
            print('2.간단소개 : %s' %desc)
            f.write('2.간단소개 : %s' %desc+'\n')
        except : 
            print('2.간단소개 : 제공된 정보 없음')
            f.write('2.간단소개 : 제공된 정보 없음'+'\n')
            desc_list.append(' ')
            
        juso = soup.find('span', '_2yqUQ').get_text().replace('\n', "").strip()
        juso_list.append(juso)
        print('3.도로명 주소 : %s' %juso)
        f.write('3.도로명 주소 : %s' %juso + '\n')
        
        try : 
            phone = soup.find('span', '_3ZA0S').get_text().replace('\n', "").strip()
            phone_list.append(phone)
            print('4.연락처 : %s' %phone)
            f.write('4.연락처 : %s' %phone+'\n')
        except : 
            print('4.연락처 : 제공된 정보 없음')
            f.write('4.연락처 : 제공된 정보 없음'+'\n')
            phone_list.append(' ')
        
        jong = soup.find('span', '_3ocDE').get_text().replace('\n', "").strip()
        jong_list.append(jong)
        print('5.업종 : %s' %jong)
        f.write('5.업종 : %s'%jong+'\n')
        
        try : 
            page = soup.find('a', '_1RUzg').get_text().replace('\n',"").strip()
            page_list.append(page)
            print('6.홈페이지 : %s' %page)
            f.write('6.홈페이지 : %s'%page+'\n')
        except : 
            print('6.홈페이지 : 제공된 정보 없음')
            f.wrtie('6.홈페이지 : 제공된 정보 없음'+'\n')
            page_list.append(' ')
        
        grade = soup.find('span', '_1Y6hi _1A8_M').find('em').get_text().replace('\n', "").strip()
        grade_list.append(grade)
        print('7.평점 : 5/%s'%grade)
        f.write('7.평점 : 5/%s' %grade+'\n')
        
        review = soup.find('span', '_1Y6hi').find('em').get_text().replace('\n',"").strip()
        review_list.append(review)
        print('8.블로그 리뷰 수 : %s'%review)
        f.write('8.블로그 리뷰 수 : %s'%review+'\n')
        
        print('-'*100)
        f.write('-'*100+'\n')
        f.write('\n')
        driver.switch_to.default_content() #
        
        #맨 아래에서 조건문 : 사진 찾고 사진 있으면 페이지다운
        if driver.find_element_by_class_name('span', 'place_blind') == True : 
            for a in range(4) : 
                driver.find_element_by_id('_pcmap_list_scroll_container').send_keys(Keys.DOWN)
                time.sleep(3)
        else : 
            driver.find_element_by_id('_pcmap_list_scroll_container').send_keys(Keys.DOWN)
            time.sleep(3)
        
        no +=1
        if no > cnt : 
            break
    if no > cnt : 
        break
    try : 
        driver.find_element_by_link_text('%s' %no).click()
        time.sleep(5)
    except : 
        break
        
f.close()
#driver.quit()



naver = pd.DataFrame()
naver['업종'] = jong_list
naver['업체명'] = title_list
naver['간단소개'] = desc_list
naver['블로그리뷰수'] = review_list
naver['고객별점'] = grade_list
naver['주소'] = juso_list
naver['연락처'] = phone_list
naver['홈페이지'] = page_list
naver.to_csv(fc_name, encoding='utf-8-sig')
naver.to_excel(fx_name)

e_time = time.time()
total_time = e_time - s_time

print('-'*100)
print('요청하신 %s 건 중 %s 건 만큼을 수집했습니다' %cnt, no-1)
print('총 소요시간은 %s초 입니다' %round(total_time, 2))
print('='*100)

        
        



업체를 조회할 키워드를 입력하세요 : 강남역 커피맛집
총 몇 건의 데이터를 조회하시겠습니까? : 60
파일 저장경로는? : 


NoSuchFrameException: Message: no such frame
  (Session info: chrome=88.0.4324.150)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.16.0 x86_64)


In [ ]:
s_time = time.time()
key_word = input('업체를 조회할 키워드를 입력하세요 : ')
cnt = int(input('총 몇 건의 데이터를 조회하시겠습니까? : '))
f_dir = input('파일 저장경로는? : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+key_word)
os.chdir(f_dir+s+'-'+key_word)

f_name = s+'-'+key_word+'.txt'
fc_name = s+'-'+key_word+'.csv'
fx_name = s+'-'+key_word+'.xlsx'


path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
driver = webdriver.Chrome(path, options=options)

In [86]:
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
driver = webdriver.Chrome(path, options=options)


driver.get('https://map.naver.com/v5/search/%EA%B0%95%EB%82%A8%EC%97%AD%20%EC%BB%A4%ED%94%BC?placeSearchOption=entry=pll%26filterId=s13479290%26from=nx%26fromNxList=true%26sessionid=w8vcfTl6IlmXf2XEMtGd7g%253D%253D&c=14140183.7974172,4508743.2143150,15,0,0,0,dh')
time.sleep(5)


#while문으로 페이지 변경 전체 반복 1번 걸기 
while True : 
    driver.switch_to.default_content()
    #항목반복 2번 걸기 
    for x in range(50) : 
        driver.switch_to.frame('searchIframe')
        try : 
            driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[1]/ul/li[%s]/div[1]/a/div[1]/div/span[1]' %x).click()
            time.sleep(5)
        except : 
            try : 
                driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[1]/ul/li[%s]/div[1]/a/div[1]/div/span' %x).click()
                time.sleep(5)
            except : 
                break
        driver.switch_to.default_content()
        
        driver.switch_to.frame('entryIframe')
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        print('-'*100)
        f.write('-'*100+'\n')
        
        title = soup.find('span', '_1_hm2').find('span', '_3XamX').get_text().replace('\n',"").strip()
        title_list.append(title)
        print('1.업체명 : %s' %title)
        f.write('1.업체명 : %s'%title+'\n')
        
        try : 
            desc = soup.find('div', 'rZaqA MFyRY').get_text().replace('\n', "").strip()
            desc_list.append(desc)
            print('2.간단소개 : %s' %desc)
            f.write('2.간단소개 : %s' %desc+'\n')
        except : 
            print('2.간단소개 : 제공된 정보 없음')
            f.write('2.간단소개 : 제공된 정보 없음'+'\n')
            desc_list.append(' ')
            
        juso = soup.find('span', '_2yqUQ').get_text().replace('\n', "").strip()
        juso_list.append(juso)
        print('3.도로명 주소 : %s' %juso)
        f.write('3.도로명 주소 : %s' %juso + '\n')
        
        try : 
            phone = soup.find('span', '_3ZA0S').get_text().replace('\n', "").strip()
            phone_list.append(phone)
            print('4.연락처 : %s' %phone)
            f.write('4.연락처 : %s' %phone+'\n')
        except : 
            print('4.연락처 : 제공된 정보 없음')
            f.write('4.연락처 : 제공된 정보 없음'+'\n')
            phone_list.append(' ')
        
        jong = soup.find('span', '_3ocDE').get_text().replace('\n', "").strip()
        jong_list.append(jong)
        print('5.업종 : %s' %jong)
        f.write('5.업종 : %s'%jong+'\n')
        
        try : 
            page = soup.find('a', '_1RUzg').get_text().replace('\n',"").strip()
            page_list.append(page)
            print('6.홈페이지 : %s' %page)
            f.write('6.홈페이지 : %s'%page+'\n')
        except : 
            print('6.홈페이지 : 제공된 정보 없음')
            f.wrtie('6.홈페이지 : 제공된 정보 없음'+'\n')
            page_list.append(' ')
        
        grade = soup.find('span', '_1Y6hi _1A8_M').find('em').get_text().replace('\n', "").strip()
        grade_list.append(grade)
        print('7.평점 : 5/%s'%grade)
        f.write('7.평점 : 5/%s' %grade+'\n')
        
        review = soup.find('span', '_1Y6hi').find('em').get_text().replace('\n',"").strip()
        review_list.append(review)
        print('8.블로그 리뷰 수 : %s'%review)
        f.write('8.블로그 리뷰 수 : %s'%review+'\n')
        
        print('-'*100)
        f.write('-'*100+'\n')
        f.write('\n')
        driver.switch_to.default_content()
        
        #맨 아래에서 조건문 : 사진 찾고 사진 있으면 페이지다운
        if driver.find_element_by_class_name('span', 'place_blind') == True : 
            for a in range(4) : 
                driver.find_element_by_id('_pcmap_list_scroll_container').send_keys(Keys.DOWN)
                time.sleep(3)
        else : 
            driver.find_element_by_id('_pcmap_list_scroll_container').send_keys(Keys.DOWN)
            time.sleep(3)
        
        no +=1
        if no > cnt : 
            break
    if no > cnt : 
        break
    try : 
        driver.find_element_by_link_text('%s' %no).click()
        time.sleep(5)
    except : 
        break
        
f.close()
#driver.quit()



naver = pd.DataFrame()
naver['업종'] = jong_list
naver['업체명'] = title_list
naver['간단소개'] = desc_list
naver['블로그리뷰수'] = review_list
naver['고객별점'] = grade_list
naver['주소'] = juso_list
naver['연락처'] = phone_list
naver['홈페이지'] = page_list
naver.to_csv(fc_name, encoding='utf-8-sig')
naver.to_excel(fx_name)

e_time = time.time()
total_time = e_time - s_time

print('-'*100)
print('요청하신 %s 건 중 %s 건 만큼을 수집했습니다' %cnt, no-1)
print('총 소요시간은 %s초 입니다' %round(total_time, 2))
print('='*100)

        
        



NoSuchFrameException: Message: no such frame
  (Session info: chrome=88.0.4324.150)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.16.0 x86_64)


In [91]:

html0 = driver.page_source
soup0 = BeautifulSoup(html0, 'html.parser')
soup0.find_all('iframe')

[]

In [78]:
driver.switch_to.default_content()

In [79]:
driver.switch_to.frame('searchIframe')

NoSuchFrameException: Message: no such frame
  (Session info: chrome=88.0.4324.150)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.16.0 x86_64)
